In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('label_guide.csv')

In [3]:
print("Duplicate Tweets:", len(df['Tweets for Form'])-len(df['Tweets for Form'].drop_duplicates()))

Duplicate Tweets: 0


In [4]:
dups = df["Tweets for Form"]
dups_listed = df[dups.isin(dups[dups.duplicated()])].sort_values("Tweets for Form")
print(dups_listed)

Empty DataFrame
Columns: [Tweet, Tweets for Form]
Index: []


In [5]:
print(len(df['Tweets for Form']))

13265
